In [0]:
!git clone https://github.com/genn-team/genn.git --branch 4.1.0

Cloning into 'genn'...
remote: Enumerating objects: 67377, done.
remote: Total 67377 (delta 0), reused 0 (delta 0), pack-reused 67377
Receiving objects: 100% (67377/67377), 207.32 MiB | 28.75 MiB/s, done.
Resolving deltas: 100% (47340/47340), done.
Note: checking out '37e7a83b1c2c5cfa158e46f4290acb24a58d6e7f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [0]:
!sudo apt-get install swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 25 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 0s (8,025 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: 

In [0]:
import os
os.environ['CUDA_PATH'] ='/usr/local/cuda'
os.chdir('genn/')
!make DYNAMIC=1 LIBRARY_DIRECTORY=`pwd`/pygenn/genn_wrapper/

make -C src/genn/genn
make[1]: Entering directory '/content/genn/src/genn/genn'
mkdir -p /content/genn/obj_dynamic/genn/genn
g++ -std=c++11 -Wall -Wpedantic -Wextra -MMD -MP -I/content/genn/include/genn/genn -I/content/genn/include/genn/third_party -fPIC -c -o /content/genn/obj_dynamic/genn/genn/synapseGroup.o synapseGroup.cc
mkdir -p /content/genn/obj_dynamic/genn/genn
g++ -std=c++11 -Wall -Wpedantic -Wextra -MMD -MP -I/content/genn/include/genn/genn -I/content/genn/include/genn/third_party -fPIC -c -o /content/genn/obj_dynamic/genn/genn/neuronModels.o neuronModels.cc
mkdir -p /content/genn/obj_dynamic/genn/genn
g++ -std=c++11 -Wall -Wpedantic -Wextra -MMD -MP -I/content/genn/include/genn/genn -I/content/genn/include/genn/third_party -fPIC -c -o /content/genn/obj_dynamic/genn/genn/initVarSnippet.o initVarSnippet.cc
mkdir -p /content/genn/obj_dynamic/genn/genn
g++ -std=c++11 -Wall -Wpedantic -Wextra -MMD -MP -I/content/genn/include/genn/genn -I/content/genn/include/genn/third_party -fP

In [0]:
!python setup.py develop

running develop
running egg_info
creating pygenn.egg-info
writing pygenn.egg-info/PKG-INFO
writing dependency_links to pygenn.egg-info/dependency_links.txt
writing requirements to pygenn.egg-info/requires.txt
writing top-level names to pygenn.egg-info/top_level.txt
writing manifest file 'pygenn.egg-info/SOURCES.txt'
writing manifest file 'pygenn.egg-info/SOURCES.txt'
running build_ext
building '_StlContainers' extension
swigging pygenn/genn_wrapper/generated/StlContainers.i to pygenn/genn_wrapper/generated/StlContainers_wrap.cpp
swig -python -c++ -relativeimport -outdir /content/genn/pygenn/genn_wrapper -I/content/genn/pygenn/genn_wrapper/include -I/content/genn/pygenn/genn_wrapper/generated -I/content/genn/pygenn/genn_wrapper/swig -py3 -o pygenn/genn_wrapper/generated/StlContainers_wrap.cpp pygenn/genn_wrapper/generated/StlContainers.i
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/pygenn
creating build/temp.linux-x86_64-3.6/pygenn/genn_wrappe

# **IMPORTS**

In [0]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt

from pygenn import genn_model, genn_wrapper 

from pygenn.genn_model import (create_custom_neuron_class,
                               create_custom_current_source_class,
                               GeNNModel)

# **LOAD DATA**

In [0]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')
import openpyxl

#Moving into the directory which contains the batch files
os.chdir("/content/drive/My Drive/Batch_Files") #Create a bacth folder 
file_names = os.listdir()

#extracting the labels corresponding to each feature vector
list_of_numbers_in_batch_files = {}
for i in range(len(file_names)):
    list_to_append = []
    if(file_names[i][-11:] == "result.xlsx"):
        print(file_names[i])
        book = openpyxl.load_workbook(file_names[i])
        sheet = book.active
        row_count = sheet.max_row
        for j in range(1,row_count):
#             print(int(sheet.cell(row=j, column=2).value))
            list_to_append.append(int(sheet.cell(row=j, column=2).value))
        list_of_numbers_in_batch_files[file_names[i][:6]] = list_to_append
print()

#extracting the data corresponding to each label per batch file
list_of_batch_files_data = {}
for i in range(len(file_names)):
    list_to_append = []
    if(file_names[i][-5:] == ".xlsx" and file_names[i][6] != "_" and file_names[i]!="y_test.xlsx"):
        print(file_names[i])
        book = openpyxl.load_workbook(file_names[i])
        sheet = book.active
        row_count = sheet.max_row
        col_count = sheet.max_column
        
        for j in range(1,row_count+1):
            list_of_data = []
            for k in range(2,col_count+1):
                list_of_data.append(sheet.cell(row=j, column=k).value)
            list_to_append.append(list_of_data)
        list_of_batch_files_data[file_names[i][:6]] = list_to_append
print()

batch_names_train = ['batch1','batch2','batch3','batch4','batch5','batch6']
batch_names_valid = ['batch7','batch8']
#getting test data
test_labels = []
test_feature_lists = []
test_features = []

for i in range(len(file_names)):
    test_features = []
    if(file_names[i] == "y_test.xlsx"):
        print(file_names[i])
        book = openpyxl.load_workbook(file_names[i])
        sheet = book.active
        row_count = sheet.max_row
        for j in range(2,row_count):
            test_labels.append(int(sheet.cell(row=j, column=2).value))
print()

for i in range(len(file_names)):
    if(file_names[i] == "X_test_normalised.xlsx"):
        print(file_names[i])
        book = openpyxl.load_workbook(file_names[i])
        sheet = book.active
        row_count = sheet.max_row
        col_count = sheet.max_column
        
        for j in range(2,row_count+1):
            test_features = []
            for k in range(2,col_count+1):
                test_features.append(sheet.cell(row=j, column=k).value)
            test_feature_lists.append(test_features)
print() 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
batch1_result.xlsx
batch2_result.xlsx
batch3_result.xlsx
batch4_result.xlsx
batch5_result.xlsx
batch6_result.xlsx
batch7_result.xlsx
batch8_result.xlsx

batch1.xlsx
batch3.xlsx
batch2.xlsx
batch5.xlsx
batch4.xlsx
batch8.xlsx
batch6.xlsx
batch7.xlsx

y_test.xlsx

X_test_normalised.xlsx



# **NETWORK**

In [0]:
# ********************************************************************************
#                      Model Definitions
# ********************************************************************************

# LIF neuron model
if_model = genn_model.create_custom_neuron_class(
    "if_model",
    param_names=["Vthr"],
    var_name_types=[("V", "scalar"), ("SpikeCount", "unsigned int")],
    sim_code="$(V) += $(Isyn) * DT;",
    reset_code="""
    $(V) = 0.0;
    $(SpikeCount)++;
    """,
    threshold_condition_code="$(V) >= $(Vthr)")

lif_e_model = genn_model.create_custom_neuron_class(
    "lif_e_model",
    param_names=["Tau","Erest","Vreset","Vthres","RefracPeriod","tauTheta"],
    var_name_types=[("V","scalar"),("RefracTime","scalar"),("theta","scalar"),("SpikeCount","unsigned int")],
    sim_code="""
    if ($(RefracTime) <= 0.0)
    {
        scalar alpha = $(Erest) + $(Isyn);
        $(V) = ($(V) - alpha) * $(ExpTC);
    }
    else
    {
        $(RefracTime) -= DT;
        $(theta) = $(theta) * $(ExpTtheta);
    }
    """,
    reset_code="""
    $(V) = $(Vreset);
    $(RefracTime) = $(RefracPeriod);
    $(SpikeCount) += 1;
    $(theta) += 1.0;
    """,
    threshold_condition_code="$(RefracTime) <= 0.0 && $(V) >= $(Vthres) + $(theta)",
    derived_params=[
        ("ExpTC", genn_model.create_dpf_class(lambda pars, dt: np.exp(-dt / pars[0]))()),
        ("ExpTtheta", genn_model.create_dpf_class(lambda pars, dt: np.exp(-dt / pars[5]))())
    ]
)

#Poission Model
# poisson_model = genn_model.create_custom_neuron_class(
#     'poisson_model',
#     var_name_types={('rate','scalar'),('timeStepToSpike','scalar')},
#     sim_code="""
#     const scalar isi = 1000.0 / $(rate);
#     if ($(timeStepToSpike) <= 0.0f) {
#         $(timeStepToSpike) += isi * $(gennrand_exponential);
#     }
#     $(timeStepToSpike) -= 1.0;
#     """,
#     threshold_condition_code="$(timeStepToSpike) <= 0.0"
# )


# Vogels_2011
stdp_model_vogels_model = genn_model.create_custom_weight_update_class(
    "vogels_2011",
    param_names=["tau", "rho", "eta", "Wmin","Wmax"],
    var_name_types=[("g", "scalar")],
    sim_code=
        """
        $(addToInSyn, $(g));
        scalar dt = $(t) - $(sT_post);
        scalar timing = exp(-dt / $(tau)) - $(rho);
        scalar newWeight = $(g) + ($(eta) * timing);
        if(newWeight < $(Wmin))
        {
            $(g) = $(Wmin);
        }
        else if(newWeight > $(Wmax))
        {
            $(g) = $(Wmax);
        }
        else
        {
            $(g) = newWeight;
        }
        """,
    learn_post_code=
        """
        scalar dt = $(t) - $(sT_pre);
        scalar timing = exp(-dt / $(tau));
        scalar newWeight = $(g) + ($(eta) * timing);
        $(g) = (newWeight > $(Wmax)) ? $(Wmax) : newWeight;
        """,

    is_pre_spike_time_required=True,
    is_post_spike_time_required=True
)


# Current source model which injects current with a magnitude specified by a state variable
cs_model = create_custom_current_source_class(
    "cs_model",
    var_name_types=[("magnitude", "scalar")],
    injection_code="$(injectCurrent, $(magnitude));")

# ********************************************************************************
#                      Parameters and Hyperparameters
# ********************************************************************************

# Population size

no_if_pop_pre = 200
no_if_pop_post = 1
simtime=1000.0



if_init = {"V": 0.0, "SpikeCount":0}
if_param = {"Vthr": 5.0}


# param_names=["Tau","Erest","Vreset","Vthres","RefracPeriod","tauTheta"],
# var_name_types=[("V","scalar"),("RefracTime","scalar"),("theta","scalar"),("SpikeCount","unsigned int")]

tau_e = 100
e_rest_e = 4.0
v_reset_e = -3.0
v_thres_e = 0.0
refrac_period_e = 5.0
e_exc = 1000
tau_theta_e = 1e7


lif_e_params = {
    "Tau":tau_e, 
    "Erest":e_rest_e,
    "Vreset":v_reset_e,
    "Vthres":v_thres_e,
    "RefracPeriod":refrac_period_e,
    "tauTheta":tau_theta_e
}

lif_e_init = {"V": genn_model.init_var("Uniform",{"min":v_reset_e,"max":v_thres_e}),
     "RefracTime":0.0, "SpikeCount":0, "theta":0.0}

weights_list = []

for i in range(10*no_if_pop_post):
    weights_of_input = np.array([])
    for j in range(no_if_pop_pre):
        weights_of_input = np.append(weights_of_input,np.array([random.uniform(0.001,0.01)]))
    weights_list.append(weights_of_input)

weights_list = np.asarray(weights_list)

stdp_params = {"tau": 20.0, "rho": 0.12, "eta":0.005, "Wmin":0.001, "Wmax": 0.01}

stim_var = {"magnitude": 0.0}

# ----------------------------------------------------------------------------
#                       Custom GeNN models
# ----------------------------------------------------------------------------

model = genn_model.GeNNModel("float","Speech_Classification_training")


# Neuron populations
# poisson_pop_post = []
# for i in range(10):
#     poisson_pop_post.append(model.add_neuron_population("poisson_pop_post"+str(i),no_if_pop_post,poisson_model,{},{'rate':100.0,'timeStepToSpike':0.0}))

neuron_layers = []
# Presynaptic population
# neuron_layers.append(model.add_neuron_population("if_pop_pre", no_if_pop_pre, if_model, if_param, if_init))
neuron_layers.append(model.add_neuron_population("if_pop_pre", no_if_pop_pre, lif_e_model, lif_e_params, lif_e_init))

# Postsynaptic population
for i in range(10):
    # neuron_layers.append(model.add_neuron_population("if_pop_post"+str(i), no_if_pop_post, if_model, if_param, if_init))
    neuron_layers.append(model.add_neuron_population("if_pop_post"+str(i), no_if_pop_post, lif_e_model, lif_e_params, lif_e_init))

# Create synaptic connections between neuronal populations
syn_io_pop = []
for i in range(10):
    syn_io_pop.append(model.add_synapse_population("syn_io_pop"+str(i),"DENSE_INDIVIDUALG",genn_wrapper.NO_DELAY,
        neuron_layers[0],neuron_layers[i+1],stdp_model_vogels_model,stdp_params,{"g":weights_list[i].flatten('C')},{},{},
            "DeltaCurr", {}, {}))

# syn_po_pop = []
# for i in range(10):
#     syn_po_pop.append(model.add_synapse_population("syn_po_pop"+str(i),"SPARSE_INDIVIDUALG",genn_wrapper.NO_DELAY,
#     poisson_pop_post[i],neuron_layers[i+1],"StaticPulse", {}, {"g": 5.0}, {}, {},
#         "DeltaCurr", {}, {},genn_model.init_connectivity("OneToOne",{})))


# Create current source to deliver input to first layers of neurons
current_input = model.add_current_source("if_pop_pre_feed", cs_model,"if_pop_pre" , {}, stim_var)

current_output = []
for i in range(10):
    current_output.append(model.add_current_source("if_pop_post_feed"+str(i),cs_model,"if_pop_post"+str(i),{},stim_var))

model.dT = 1.0
print("Building Model")
model.build()
print("Loading Model")
model.load()

weight_initial = []
for i in range(10):
    weight_initial.append(syn_io_pop[i].get_var_values('g'))

spike_number_record_train = np.zeros((10*no_if_pop_post,10))
layer_voltages_V = [l.vars["V"].view for l in neuron_layers]
output_spike_count = []
for i in range(10):
    output_spike_count.append(neuron_layers[i+1].vars["SpikeCount"].view)
current_input_magnitude = current_input.vars["magnitude"].view

current_output_magnitude = []
for i in range(10):
    current_output_magnitude.append(current_output[i].vars["magnitude"].view)

for batch in batch_names_train:

    features_list = list_of_batch_files_data[batch]
    digit_list = list_of_numbers_in_batch_files[batch]

    for (feature, correct_digit) in zip(features_list,digit_list):
        model.timestep = 0
        while model.timestep < simtime:

            if model.timestep==0:
                current_input_magnitude[:] = feature
                model.push_var_to_device("if_pop_pre_feed", "magnitude")
                for i in range(10):
                    # neuron_view = poisson_pop_post[i].vars['timeStepToSpike'].view
                    # neuron_view[:] = 0.0
                    if correct_digit==i:
                        # neuron_view = poisson_pop_post[i].vars['rate'].view
                        # neuron_view[:] = 500.0
                        current_output_magnitude[i][:] = sum(feature)/len(feature)
                    else:
                        # neuron_view = poisson_pop_post[i].vars['rate'].view
                        # neuron_view[:] = 0.0
                        current_output_magnitude[i][:] = -1.0
                    # model.push_state_to_device('poisson_pop_post'+str(i))
                    model.push_var_to_device("if_pop_post_feed"+str(i),"magnitude")
                for l, v in zip(neuron_layers, layer_voltages_V):
                    # Manually 'reset' voltage
                    v[:] = 0.0

                    # Upload
                    model.push_var_to_device(l.name, "V")

                # Zero spike count
                for i in range(10):
                    output_spike_count[i][:] = 0
                    model.push_var_to_device(neuron_layers[i+1].name, "SpikeCount")

            
            model.step_time()
            if model.timestep == simtime-1:
                for i in range(10):
                    model.pull_var_from_device("if_pop_post"+str(i),"SpikeCount")
                    spike_number_view = neuron_layers[i+1].vars["SpikeCount"].view
                    spike_number_record_train[i,correct_digit] += spike_number_view
                    neuron_layers[i+1].set_var("SpikeCount",0)
                print("Spike Train: {} Label: {}".format(spike_number_record_train,correct_digit))

# Retrieving Final weights
weight_final = []
for i in range(10):
    model.pull_var_from_device('syn_io_pop'+str(i),'g')
    weight_final.append(syn_io_pop[i].get_var_values('g'))


#printing spikes
print(spike_number_record_train)


Streaming output truncated to the last 5000 lines.
 [ 1.  0. 34. 17.  9. 35.  2.  0.  0.  0.]
 [ 2. 54.  0. 16.  7. 15.  3.  0.  0.  1.]
 [ 1. 53. 20.  0.  7. 15.  2.  0.  0.  0.]
 [ 0. 53. 19. 10.  0. 15.  1.  0.  0.  0.]
 [ 0. 58. 24. 10.  5.  0.  1.  0.  0.  0.]
 [ 0. 53. 17.  9.  4. 15.  0.  0.  0.  0.]
 [ 0. 53. 17.  9.  4. 15.  0.  0.  0.  0.]
 [ 0. 53. 17.  9.  4. 15.  0.  0.  0.  0.]
 [ 0. 53. 17.  9.  4. 15.  0.  0.  0.  0.]] Label: 6
Spike Train: [[ 0. 53. 17.  9.  4. 15.  0.  0.  0.  0.]
 [ 1.  0. 34. 17.  9. 35.  2.  0.  0.  0.]
 [ 2. 54.  0. 16.  7. 15.  3.  0.  0.  1.]
 [ 1. 53. 20.  0.  7. 15.  2.  0.  0.  0.]
 [ 0. 53. 19. 10.  0. 15.  1.  0.  0.  0.]
 [ 0. 58. 24. 10.  5.  0.  1.  0.  0.  0.]
 [ 0. 53. 17.  9.  4. 15.  0.  0.  0.  0.]
 [ 0. 53. 17.  9.  4. 15.  0.  0.  0.  0.]
 [ 0. 53. 17.  9.  4. 15.  0.  0.  0.  0.]
 [ 0. 53. 17.  9.  4. 15.  0.  0.  0.  0.]] Label: 3
Spike Train: [[ 0. 53. 17.  9.  4. 15.  0.  0.  0.  0.]
 [ 1.  0. 34. 17.  9. 35.  2.  0.  0.  0.]


In [0]:
# Printing Initial weights
print("Weight Initial")
print(weight_initial)
# Printing final weights
print("Weight Final")
print(weight_final)

Weight Initial
[array([0.00443752, 0.00446568, 0.00548842, 0.00420619, 0.00366117,
       0.00129099, 0.0011319 , 0.00383745, 0.00330469, 0.00283297,
       0.00169298, 0.00878097, 0.00252067, 0.00538234, 0.00125276,
       0.00899639, 0.00756368, 0.00517366, 0.00700423, 0.003528  ,
       0.00143392, 0.0057451 , 0.00848848, 0.00820148, 0.00247807,
       0.00187791, 0.00774985, 0.00233563, 0.00125254, 0.0034061 ,
       0.00445436, 0.00629734, 0.00147139, 0.00858869, 0.0092014 ,
       0.00388455, 0.00573472, 0.00455758, 0.00662066, 0.00680102,
       0.00949946, 0.00104531, 0.00955205, 0.00289853, 0.00728088,
       0.00645019, 0.00316235, 0.00916151, 0.00539384, 0.00820574,
       0.0052967 , 0.00680515, 0.00312548, 0.00983505, 0.00574876,
       0.00943897, 0.00181704, 0.00483343, 0.00183097, 0.00557293,
       0.00520653, 0.00386216, 0.00808782, 0.0034082 , 0.00130273,
       0.00887652, 0.00955828, 0.00680383, 0.00289635, 0.00974342,
       0.00576725, 0.00890451, 0.00653295, 0.0

In [0]:
for i in range(10):
    weight_final[i] = np.asarray(weight_final[i])
weight_final = np.asarray(weight_final)

weight_final

array([[0.0090233 , 0.00111222, 0.00502473, ..., 0.00230315, 0.00936129,
        0.00628389],
       [0.0043095 , 0.00376117, 0.00576834, ..., 0.00846572, 0.00213074,
        0.00838349],
       [0.00684641, 0.0071308 , 0.00704745, ..., 0.00861678, 0.00365179,
        0.00266342],
       ...,
       [0.00973481, 0.00792613, 0.00154241, ..., 0.00285441, 0.00782927,
        0.00588084],
       [0.00219786, 0.0027668 , 0.00357918, ..., 0.00208227, 0.0019755 ,
        0.00341811],
       [0.00527296, 0.00470356, 0.00263518, ..., 0.00432407, 0.0078596 ,
        0.00658682]], dtype=float32)

# **Classification on Validation Dataset**

In [0]:
model = genn_model.GeNNModel("float","Speech_Classification_valid")

# Neuron populations for testing purposes
neuron_layers_post = []
# IF Presynaptic population
# neuron_layers_post.append(model.add_neuron_population("test_if_pop_pre", no_if_pop_pre, if_model, if_param, if_init))
neuron_layers_post.append(model.add_neuron_population("test_if_pop_pre", no_if_pop_pre, lif_e_model, lif_e_params, lif_e_init))


# IF Postsynaptic population
for i in range(10):
    # neuron_layers_post.append(model.add_neuron_population("test_if_pop_post"+str(i), no_if_pop_post, if_model, if_param, if_init))
  neuron_layers_post.append(model.add_neuron_population("test_if_pop_post"+str(i), no_if_pop_post, lif_e_model, lif_e_params, lif_e_init))

# Create synaptic connections between neuronal populations
syn_io_pop_test = []
for i in range(10):
    syn_io_pop_test.append(model.add_synapse_population("syn_io_pop_test"+str(i),"DENSE_INDIVIDUALG",genn_wrapper.NO_DELAY,
        neuron_layers_post[0],neuron_layers_post[i+1],"StaticPulse",{},{"g":weight_final[i].flatten('C')},{},{},
            "DeltaCurr", {}, {}))


# Create current source to deliver input to first layers of neurons
current_input = model.add_current_source("if_pop_pre_feed", cs_model,"test_if_pop_pre" , {}, stim_var)


model.dT = 1.0
print("Building Model")
model.build()
print("Loading Model")
model.load()

layer_voltages_V = [l.vars["V"].view for l in neuron_layers_post]
output_spike_count = []
for i in range(10):
    output_spike_count.append(neuron_layers_post[i+1].vars["SpikeCount"].view)
current_input_magnitude = current_input.vars["magnitude"].view
spike_number_record = np.zeros((10))
num_correct = 0
total_num = 0

for batch in batch_names_valid:

    features_list = list_of_batch_files_data[batch]
    digit_list = list_of_numbers_in_batch_files[batch]

    for (feature, correct_digit) in zip(features_list,digit_list):
        total_num+=1
        model.timestep = 0
        while model.timestep < simtime:
            if model.timestep==0:
                current_input_magnitude[:] = feature
                model.push_var_to_device("if_pop_pre_feed", "magnitude")

                for l, v in zip(neuron_layers_post, layer_voltages_V):
                    # Manually 'reset' voltage
                    v[:] = 0.0

                    # Upload
                    model.push_var_to_device(l.name, "V")

                # Zero spike count
                for i in range(10):
                    output_spike_count[i][:] = 0
                    model.push_var_to_device(neuron_layers_post[i+1].name, "SpikeCount")

            
            model.step_time()

            if model.timestep == simtime-1:
                for i in range(10):
                    model.pull_var_from_device("test_if_pop_post"+str(i),"SpikeCount")
                    spike_number_view = neuron_layers_post[i+1].vars["SpikeCount"].view
                    spike_number_record[i] += spike_number_view
                    neuron_layers_post[i+1].set_var("SpikeCount",0)
                
                max_ind = np.argmax(spike_number_record)
                print(spike_number_record)
                
        print("Predicted Label: {} Label: {}".format(max_ind,correct_digit))

        if correct_digit==max_ind:
            num_correct+=1
        spike_number_record = np.zeros((10))

print("Validation Accuracy:")
print(num_correct/total_num)


Building Model
Loading Model
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 6
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 7
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 5
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 6
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 2
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 7
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 6
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 3
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 1
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 8
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 9
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 2
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 9
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 2
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 6
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Label: 0 Label: 1
[0. 0. 0. 0

# **Classification on Test Dataset**

In [0]:
model = genn_model.GeNNModel("float","Speech_Classification_test")

# Neuron populations for testing purposes
neuron_layers_post = []
# IF Presynaptic population
# neuron_layers_post.append(model.add_neuron_population("test_if_pop_pre", no_if_pop_pre, if_model, if_param, if_init))
neuron_layers_post.append(model.add_neuron_population("test_if_pop_pre", no_if_pop_pre, lif_e_model, lif_e_params, lif_e_init))


# IF Postsynaptic population
for i in range(10):
    # neuron_layers_post.append(model.add_neuron_population("test_if_pop_post"+str(i), no_if_pop_post, if_model, if_param, if_init))
    neuron_layers_post.append(model.add_neuron_population("test_if_pop_post"+str(i), no_if_pop_post, lif_e_model, lif_e_params, lif_e_init))

# Create synaptic connections between neuronal populations
syn_io_pop_test = []
for i in range(10):
    syn_io_pop_test.append(model.add_synapse_population("syn_io_pop_test"+str(i),"DENSE_INDIVIDUALG",genn_wrapper.NO_DELAY,
        neuron_layers_post[0],neuron_layers_post[i+1],"StaticPulse",{},{"g":weight_final[i].flatten('C')},{},{},
            "DeltaCurr", {}, {}))


# Create current source to deliver input to first layers of neurons
current_input = model.add_current_source("if_pop_pre_feed", cs_model,"test_if_pop_pre" , {}, stim_var)


model.dT = 1.0
print("Building Model")
model.build()
print("Loading Model")
model.load()


layer_voltages_V = [l.vars["V"].view for l in neuron_layers_post]
output_spike_count = []
for i in range(10):
    output_spike_count.append(neuron_layers_post[i+1].vars["SpikeCount"].view)
current_input_magnitude = current_input.vars["magnitude"].view
spike_number_record = np.zeros((10))
num_correct = 0
total_num = 0

for (feature, correct_digit) in zip(test_feature_lists,test_labels):
    total_num+=1
    model.timestep = 0
    while model.timestep < simtime:
        if model.timestep==0:
            current_input_magnitude[:] = feature
            model.push_var_to_device("if_pop_pre_feed", "magnitude")

            for l, v in zip(neuron_layers_post, layer_voltages_V):
                # Manually 'reset' voltage
                v[:] = 0.0

                # Upload
                model.push_var_to_device(l.name, "V")

            # Zero spike count
            for i in range(10):
                output_spike_count[i][:] = 0
                model.push_var_to_device(neuron_layers_post[i+1].name, "SpikeCount")

        
        model.step_time()

        if model.timestep == simtime-1:
            for i in range(10):
                model.pull_var_from_device("test_if_pop_post"+str(i),"SpikeCount")
                spike_number_view = neuron_layers_post[i+1].vars["SpikeCount"].view
                spike_number_record[i] += spike_number_view
                neuron_layers_post[i+1].set_var("SpikeCount",0)
            
            max_ind = np.argmax(spike_number_record)
            print(spike_number_record)
            
    print("Predicted Label: {} Label: {}".format(max_ind,correct_digit))

    if correct_digit==max_ind:
        num_correct+=1
    spike_number_record = np.zeros((10))

print("Test Accuracy:")
print(num_correct/total_num)